In [0]:
!pip install boto3 faker

In [0]:
import json
import boto3
import random
import time
from datetime import datetime, timezone
from faker import Faker

AWS_ACCESS_KEY = dbutils.secrets.get(scope="project_secrets", key="aws_access_key")
AWS_SECRET_KEY = dbutils.secrets.get(scope="project_secrets", key="aws_secret_key")

BUCKET_NAME = dbutils.widgets.get("p_bucket_name")

class FleetTelemetryGenerator:
    def __init__(self):
        self.fake = Faker("pt_BR")
        self.s3_client = boto3.client(
            "s3",
            aws_access_key_id=AWS_ACCESS_KEY,
            aws_secret_access_key=AWS_SECRET_KEY,
            region_name="us-east-1"
        )

    def generate_record(self):
        return {
            "event_id": self.fake.uuid4(),
            "truck_id": random.randint(1, 10),
            "timestamp": datetime.now(timezone.utc).isoformat(),
            "gps_latitude": float(self.fake.latitude()),
            "gps_longitude": float(self.fake.longitude()),
            "engine_rpm": random.randint(1500, 3500),
            "speed_kmh": random.randint(60, 120),
            "fuel_level": random.randint(10, 100),
            "engine_temp": random.randint(80, 115)
        }

    def upload_batch(self):
        data = [self.generate_record() for _ in range(50)]
        payload = "\n".join([json.dumps(d) for d in data])
        
        now = datetime.now()
        file_path = f"telemetry_data/year={now.year}/month={now.month:02d}/telemetry_{now.strftime('%H%M%S')}.json"
        
        try:
            self.s3_client.put_object(Bucket=BUCKET_NAME, Key=file_path, Body=payload)
            print(f"Upload completed: {file_path} | {datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M:%S')}")
        except Exception as e:
            print(f"Error: {e}")

if __name__ == "__main__":
    gen = FleetTelemetryGenerator()
    while True:
        gen.upload_batch()
        time.sleep(60)